## Calculating tobin's q of S&P 500 firms Using tobin's Q


### Installing some modules

In [1]:
#%pip install lxml
#import pandas as pd
#%pip install yfinance

In [4]:
# sp500_liabilities_fixed.py

import yfinance as yf
import pandas as pd
import requests
from io import StringIO

# Step 1: Get the list of S&P 500 tickers from Wikipedia (avoid 403 error)
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Failed to fetch S&P 500 list. HTTP {response.status_code}")

sp500_df = pd.read_html(StringIO(response.text))[0]
sp500_df.columns = [c.strip() for c in sp500_df.columns]
tickers = sp500_df["Symbol"].tolist()

print(f"✅ Retrieved {len(tickers)} S&P 500 tickers successfully.")

# Step 2: Example - Fetch total liabilities for GOOGL
ticker = "GOOGL"
try:
    t = yf.Ticker(ticker)
    info = t.info or {}
    liab = info.get("totalLiab", None)

    if liab is not None:
        print(f"Total Liabilities of {ticker}: {liab:,}")
    else:
        print(f"⚠️ Could not find total liabilities data for {ticker}.")
except Exception as e:
    print(f"Error retrieving data for {ticker}: {e}")


✅ Retrieved 503 S&P 500 tickers successfully.
⚠️ Could not find total liabilities data for GOOGL.


In [8]:
# sp500_tobins_q_fixed.py

import yfinance as yf
import pandas as pd
import requests
from io import StringIO
import time

# Optional: Set pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# Step 1: Get the list of S&P 500 tickers from Wikipedia (avoid HTTP 403)
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Failed to fetch S&P 500 list. HTTP {response.status_code}")

sp500_df = pd.read_html(StringIO(response.text))[0]
sp500_df.columns = [c.strip() for c in sp500_df.columns]
tickers = sp500_df["Symbol"].tolist()

print(f"✅ Retrieved {len(tickers)} S&P 500 tickers successfully.\n")

# Step 2: Collect key data
def collect_data(ticker):
    """
    Collects key financial data for a given ticker.
    Returns a dictionary containing market cap, total assets, and total liabilities.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info or {}
        market_cap = info.get("marketCap", None)

        bs = stock.balance_sheet
        if bs is None or bs.empty or market_cap is None:
            print(f"⚠️ Missing financials for {ticker}")
            return None

        latest_date = bs.columns[0]

        # Try to find Total Assets (common variations)
        asset_candidates = [
            "Total Assets", "total assets"
        ]
        assets = None
        for key in asset_candidates:
            if key in bs.index:
                assets = bs.loc[key, latest_date]
                break

        # Try to find Total Liabilities (common variations)
        liab_candidates = [
            "Total Liabilities Net Minority Interest",
            "Total Liab", "total liabilities", "Total Liabilities"
        ]
        liab = None
        for key in liab_candidates:
            if key in bs.index:
                liab = bs.loc[key, latest_date]
                break

        if assets is None or liab is None or assets == 0:
            print(f"⚠️ Missing balance sheet data for {ticker}")
            return None

        return {
            "market_cap": market_cap,
            "total_assets": assets,
            "total_liab": liab
        }

    except Exception as e:
        print(f"❌ Error processing {ticker}: {e}")
        return None

# Step 3: Compute Tobin’s Q
def calculate_tobins_q(data):
    """
    Calculates Tobin's Q = (Market Cap + Total Liabilities) / Total Assets
    """
    if data is None or data["total_assets"] == 0:
        return None
    return (data["market_cap"] + data["total_liab"]) / data["total_assets"]

# Step 4: Loop through tickers and compute results
results = []
for i, ticker in enumerate(tickers):
    data = collect_data(ticker)
    q_value = calculate_tobins_q(data)
    if q_value is not None:
        results.append({
            "Ticker": ticker,
            "Tobin's Q": q_value
        })
    print(f"[{i+1}/{len(tickers)}] {ticker} processed.")
    time.sleep(1)  # be polite to Yahoo API

# Step 5: Create and save DataFrame
df_results = pd.DataFrame(results)
print("\n✅ Completed! Preview:\n", df_results.head())

df_results.to_csv("sp500_tobins_q.csv", index=False)
print("\n📁 Saved to sp500_tobins_q.csv")


✅ Retrieved 503 S&P 500 tickers successfully.

[1/503] MMM processed.
[2/503] AOS processed.
[3/503] ABT processed.
[4/503] ABBV processed.
[5/503] ACN processed.
[6/503] ADBE processed.
[7/503] AMD processed.
[8/503] AES processed.
[9/503] AFL processed.
[10/503] A processed.
[11/503] APD processed.
[12/503] ABNB processed.
[13/503] AKAM processed.
[14/503] ALB processed.
[15/503] ARE processed.
[16/503] ALGN processed.
[17/503] ALLE processed.
[18/503] LNT processed.
[19/503] ALL processed.
[20/503] GOOGL processed.
[21/503] GOOG processed.
[22/503] MO processed.
[23/503] AMZN processed.
[24/503] AMCR processed.
[25/503] AEE processed.
[26/503] AEP processed.
[27/503] AXP processed.
[28/503] AIG processed.
[29/503] AMT processed.
[30/503] AWK processed.
[31/503] AMP processed.
[32/503] AME processed.
[33/503] AMGN processed.
[34/503] APH processed.
[35/503] ADI processed.
[36/503] AON processed.
[37/503] APA processed.
[38/503] APO processed.
[39/503] AAPL processed.
[40/503] AMAT pr

In [ ]:
print(sp500_df.columns.tolist())


['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']


In [ ]:
merged_df.sort_values(by=["Tobin's Q"])

,Ticker,Full Name,Sector,Industry,Date Added,Founded (Wiki),Market Cap,Founded (YF),Total Assets,Total Liabilities,Tobin's Q
254,IVZ,Invesco,Financials,Asset Management & Custody Banks,2008-08-21,1935,6495648768,None,2.700890e+10,1.134010e+10,0.660366
14,ARE,Alexandria Real Estate Equities,Real Estate,Office REITs,2017-03-20,1994,12904978432,None,3.752745e+10,1.512899e+10,0.747026
470,VTRS,Viatris,Health Care,Pharmaceuticals,2004-04-23,1961,10281406464,None,4.150090e+10,2.286540e+10,0.798701
360,PARA,Paramount Global,Communication Services,Movies & Entertainment,1994-09-30,2019 (Paramount Pictures 1912),8302555136,None,4.617200e+10,2.939000e+10,0.816351
13,ALB,Albemarle Corporation,Materials,Specialty Chemicals,2016-07-01,1994,7210854400,None,1.660965e+10,6.409961e+09,0.820054
276,KHC,Kraft Heinz,Consumer Staples,Packaged Foods & Meats,2015-07-06,2015 (1869),33825572864,None,8.828700e+10,3.896200e+10,0.824443
481,WBD,Warner Bros. Discovery,Communication Services,Broadcasting,2022-04-11,2022 (Warner Bros. 1923),21347508224,None,1.045600e+11,6.962200e+10,0.870022
201,BEN,Franklin Resources,Financials,Asset Management & Custody Banks,1998-04-30,1947,10508380160,None,3.246450e+10,1.789970e+10,0.875051
318,MHK,Mohawk Industries,Consumer Discretionary,Home Furnishings,2013-12-23,1878,6462862336,None,1.277860e+10,5.221700e+09,0.914385
326,MOS,Mosaic Company (The),Materials,Fertilizers & Agricultural Chemicals,2011-09-26,2004 (1865 / 1909),9678688256,None,2.292400e+10,1.130930e+10,0.915547
